# XOR-Net

A simple neural network that solves XOR operation

### 📐 Data Setup

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(hash("piero"))

# Define dataset
left  = torch.tensor([0, 0, 1, 0, 1, 1]).unsqueeze(1).repeat(2, 1)  # (8, 1)
right = torch.tensor([0, 1, 0, 1, 0, 1]).unsqueeze(1).repeat(2, 1)  # (8, 1)
X_train = torch.cat((left, right), dim=1).float()  # (4, 2)
y_train = (left ^ right).float()

### 🧠 Model

In [ ]:
from typing import Any


class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(2, 2)  # 2 input features, 2 hidden units
        self.fc2 = nn.Linear(2, 1)  # 2 hidden units, 1 output

    def forward(self, x) -> torch.Tensor:
        x = torch.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

    # This is only done to force intelliSense to recognize the return type as torch.Tensor
    def __call__(self, *args: Any, **kwds: Any) -> torch.Tensor:
        x = super().__call__(*args, **kwds)
        if isinstance(x, torch.Tensor):
            return x
        raise TypeError(f"Expected torch.Tensor, got {type(x)}")

### ⚙️ Training

In [ ]:
model = SimpleNet()
loss_fn = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(1000):
    y_pred = model(X_train)
    loss = loss_fn(y_pred, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print("Final predictions:", y_pred.detach().round().squeeze())

### 🧪 Create Test Set

In [ ]:
test_size = 10
lefts = torch.randint(0, 2, (test_size, 1))
rights = torch.randint(0, 2, (test_size, 1))
X_test = torch.cat((lefts, rights), dim=1).float()  # (test_size, 2)
y_test = (lefts ^ rights).float()

### ✅ Test Accuracy

In [ ]:
with torch.no_grad():  # Disable gradient tracking for inference
    y_pred = model(X_test)
    print("Test predictions (raw):", y_pred.squeeze())
    y_pred_labels = (y_pred >= 0.5).float()  # Threshold at 0.5
    print("Test predictions:", y_pred_labels.squeeze())
    correct = (y_pred_labels == y_test).sum().item()
    accuracy = correct / len(y_test)

print(f"Accuracy: {accuracy*100:.2f}%")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Convert to CPU lists
y_true = y_test.cpu().tolist()
y_pred = y_pred_labels.cpu().tolist()

# Compute matrix
cm = confusion_matrix(y_true, y_pred)

# Plot
plt.figure(figsize=(4, 3))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["0s", "1s"], yticklabels=["0s", "1s"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.tight_layout()
plt.show()